In [ ]:
from labda import Subject
import pandas as pd

sbj = Subject.from_parquet("temp.parquet")
crs = sbj.metadata.crs
sampling_frequency = sbj.metadata.sampling_frequency

In [ ]:
from pykalman import KalmanFilter
import numpy as np
from PIL import ImageColor
import pydeck as pdk
from labda.spatial.utils import df_to_gdf

df = sbj.df.copy()

temp = df[["latitude_interpolated", "longitude_interpolated"]].copy()
temp.rename(
    columns={
        "latitude_interpolated": "latitude",
        "longitude_interpolated": "longitude",
    },
    inplace=True,
)
temp.reset_index(drop=False, inplace=True)

dt = 15.0
transition_matrix = np.array([[1, 0, dt, 0], [0, 1, 0, dt], [0, 0, 1, 0], [0, 0, 0, 1]])
observation_matrix = np.array([[1, 0, 0, 0], [0, 1, 0, 0]])
process_covariance = np.diag([0.1, 0.1, 0.01, 0.01])
observation_covariance = np.diag([0.5, 0.5])
initial_state_mean = [temp["longitude"].iloc[0], temp["latitude"].iloc[0], 0, 0]
initial_state_covariance = np.eye(4) * 1.0

kf = KalmanFilter(
    transition_matrices=transition_matrix,
    observation_matrices=observation_matrix,
    initial_state_mean=initial_state_mean,
    initial_state_covariance=initial_state_covariance,
    transition_covariance=process_covariance,
    observation_covariance=observation_covariance,
)

coordinates = temp[["longitude", "latitude"]].values
filtered_state_estimates = kf.filter(coordinates)[0]

temp["kalman_longitude"] = filtered_state_estimates[:, 0]
temp["kalman_latitude"] = filtered_state_estimates[:, 1]

kalman = df_to_gdf(
    temp, crs, longitude="kalman_longitude", latitude="kalman_latitude"
).to_crs("EPSG:4326")

layer = pdk.Layer(
    "GeoJsonLayer",
    kalman,
    filled=True,
    get_fill_color=ImageColor.getrgb("red"),
    get_radius=10,
    pickable=True,
)

layers = [layer]

center = kalman["geometry"].union_all().centroid
view_state = pdk.ViewState(longitude=center.x, latitude=center.y, zoom=15)

map = pdk.Deck(
    layers=layers,
    initial_view_state=view_state,
    tooltip={"html": "{tooltip}"},  # type: ignore
    # map_style=pdk.map_styles.CARTO_LIGHT,
)

map